## Check HW








In [ ]:
!lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              2
On-line CPU(s) list: 0,1
Thread(s) per core:  2
Core(s) per socket:  1
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               79
Model name:          Intel(R) Xeon(R) CPU @ 2.20GHz
Stepping:            0
CPU MHz:             2199.998
BogoMIPS:            4399.99
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            256K
L3 cache:            56320K
NUMA node0 CPU(s):   0,1
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_sin

In [ ]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:            12G        638M         10G        1.3M        1.9G         11G
Swap:            0B          0B          0B


In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [ ]:
!nvidia-smi

Fri Sep 23 08:08:13 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Install the environment

In [ ]:
#!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

In [ ]:
!rm -rf master.zip* nvcc4jupyter-master*
!wget https://github.com/andreinechaev/nvcc4jupyter/archive/refs/heads/master.zip
!unzip master.zip
!cd nvcc4jupyter-master; ls; python setup.py install

--2022-09-23 08:31:01--  https://github.com/andreinechaev/nvcc4jupyter/archive/refs/heads/master.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/andreinechaev/nvcc4jupyter/zip/refs/heads/master [following]
--2022-09-23 08:31:01--  https://codeload.github.com/andreinechaev/nvcc4jupyter/zip/refs/heads/master
Resolving codeload.github.com (codeload.github.com)... 20.205.243.165
Connecting to codeload.github.com (codeload.github.com)|20.205.243.165|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5201 (5.1K) [application/zip]
Saving to: ‘master.zip’

master.zip          100%[===================>]   5.08K  --.-KB/s    in 0s      

2022-09-23 08:31:02 (58.3 MB/s) - ‘master.zip’ saved [5201/5201]

Archive:  master.zip
aac710a35f52bb78ab34d2e52517237941399eff
   creating: nvcc4jupyter-master/
 extracting

In [ ]:
%load_ext nvcc_plugin

The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


## Run a program (using Google overpowered GPUs)



### Simple mode, everything on the same file

This is the only part of the notebook that really uses the extension nvcc_plugin

In [ ]:
%%cu

#include <stdio.h>
#include <cuda.h>
#include <cuda_runtime.h>

__global__ void mykernel(void) {

}

int main(void) {
		mykernel<<<1,1>>>();
		printf("Hello world\n");
		return 0;
}

Hello world



In [ ]:
%%cu
#include <cuda.h>
#include <cuda_runtime.h>
#include <stdio.h>

__global__ void add(int *a, int *b, int *c) {
    *c = *a + *b;
}

int main(void) {
    int a, b, c;				    // host copies of a, b, c
	  int *d_a, *d_b, *d_c;		// device copies of a, b, c
	  int size = sizeof(int);

	  // Allocate space for device copies of a, b, c
	  cudaMalloc((void **)&d_a, size);
	  cudaMalloc((void **)&d_b, size);
	  cudaMalloc((void **)&d_c, size);

	  // Setup input values
	  a = 2;
	  b = 7;

	  // Copy inputs to device
	  cudaMemcpy(d_a, &a, size, cudaMemcpyHostToDevice);
	  cudaMemcpy(d_b, &b, size, cudaMemcpyHostToDevice);

	  // Launch add() kernel on GPU
	  add<<<1,1>>>(d_a, d_b, d_c);

	  // Copy result back to host
	  cudaMemcpy(&c, d_c, size, cudaMemcpyDeviceToHost);

	  // Show result
	  printf("%d + %d = %d", a, b, c);

	  // Cleanup
	  cudaFree(d_a); cudaFree(d_b); cudaFree(d_c);
	  return 0;
}

2 + 7 = 9


### Running code in multiple source files

In [ ]:
!mkdir cuda
!rm -rf cuda/*

mkdir: cannot create directory ‘cuda’: File exists


In [ ]:
%%writefile cuda/kernel.cu
#include <cuda.h>

__global__ void add(int *a, int *b, int *c) {
    *c = *a + *b;
}

Writing cuda/kernel.cu


In [ ]:
%%writefile cuda/main.cu
#include <cuda.h>
#include <cuda_runtime.h>
#include <stdio.h>

__global__ void add(int *a, int *b, int *c);

int main(void) {
	int a, b, c;				    // host copies of a, b, c
	int *d_a, *d_b, *d_c;		// device copies of a, b, c
	int size = sizeof(int);

	// Allocate space for device copies of a, b, c
	cudaMalloc((void **)&d_a, size);
	cudaMalloc((void **)&d_b, size);
	cudaMalloc((void **)&d_c, size);

	// Setup input values
	a = 2;
	b = 7;

	// Copy inputs to device
	cudaMemcpy(d_a, &a, size, cudaMemcpyHostToDevice);
	cudaMemcpy(d_b, &b, size, cudaMemcpyHostToDevice);

	// Launch add() kernel on GPU
	add<<<1,1>>>(d_a, d_b, d_c);

	// Copy result back to host
	cudaMemcpy(&c, d_c, size, cudaMemcpyDeviceToHost);

	// Show result
	printf("%d + %d = %d", a, b, c);

	// Cleanup
	cudaFree(d_a); cudaFree(d_b); cudaFree(d_c);
	return 0;
}

Writing cuda/main.cu


In [ ]:
!nvcc cuda/*.cu -o cuda/program
!./cuda/program
!rm -rf ./cuda/program

2 + 7 = 9

## Mount your Google Drive

In [ ]:
# Mount Google Drive
from google.colab import drive
#drive.flush_and_unmount()
drive.mount('/content/gdrive')

In [ ]:
# Copy source code to your folder
!cp /content/gdrive/MyDrive/XXX/* cuda/
# Create soft links
!ln -s /content/gdrive/MyDrive/XXXX cuda/XXXX
# List folder
!ls -la cuda/